In [ ]:
from accelerate.test_utils.scripts.test_script import training_check
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from bitnet_selfdistil import lm_losses_calculator, ReLoRAConfig, ReLoRAEvents, ReloraTrainer, BitLinearWithLoRA
from bitnet_selfdistill_utils import phi3_full_gradient_checkpoint_enable
from torch.optim import AdamW

In [ ]:
MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"
DEVICE = "cuda:0"
LORA_RANK = 128

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map=DEVICE,
)

In [ ]:
def check_generation(model, tokenizer):
    with torch.no_grad():
        input_ids = tokenizer.apply_chat_template(
            [
                {
                    "role": "user",
                    "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
                }
            ],
            return_tensors="pt",
            add_generation_prompt=True
        ).to(device=DEVICE)
        generation_output = model.generate(input_ids=input_ids,
                                           return_dict_in_generate=True,
                                           output_scores=True,
                                           max_length=100)
        response = tokenizer.decode(generation_output.sequences[0][input_ids.shape[1]:])
        print(response)

In [ ]:
check_generation(model, tokenizer)

In [ ]:
def _global_lr(step):
    if step < 2000:
        return step / 2000
    else:
        return 1.0

In [ ]:
def _step_end(step, optimizer, losses, loss):
    if step % 50 == 0:
        print(f"STEP {step}")
        for loss_name, loss_value in losses.items():
            print(f"{loss_name}: {loss_value.item():.4f}")

In [ ]:
def _chunk_end(chunk, step):
    print(f"CHUNK {chunk} FINISHED AT STEP {step}")

In [ ]:
relora_config = ReLoRAConfig(
    blacklisted_modules=["lm_head"],
    lora_rank=128,
    optimizer_type=AdamW,
    optimizer_kwargs={
        "lr": 1e-4,
    },
    reset_steps=1000,
    chunk_warmup_steps=100,
    lr_global=_global_lr,
)

In [ ]:
relora_events = ReLoRAEvents(
    on_step_end=_step_end,
    on_chunk_end=_chunk_end,
)

In [ ]:
model = phi3_full_gradient_checkpoint_enable(model)

In [ ]:
trainer = ReloraTrainer(
    model=model,
    relora_config=relora_config,
    events=relora_events,
    losses_calculator=lm_losses_calculator(4096),
    max_steps=20000,
    model_kwargs={
        "output_hidden_states": True,
    }
)

In [ ]:
batch = {
    "input_ids": tokenizer.apply_chat_template(
        [
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
            {
                "role": "user",
                "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
            },
            {
                "role": "assistant",
                "content": "Certainly! Bananas and dragonfruits can be combined in a variety of delicious and creative ways. Here are some ideas:\n" + \
                           "- Blended Smoothie:\n" + \
                           "  Peel and cut both fruits into chunks and blend them with some yogurt or coconut milk for creaminess. Add a scoop of protein powder or a spoonful of peanut butter for extra protein and flavor."
            },
        ],
        return_tensors="pt",
        add_generation_prompt=True
    ).to(device=DEVICE)
}
batch["labels"] = batch["input_ids"].clone()

In [ ]:
batch["input_ids"].shape

In [ ]:
trainer.patch.init_lora("lora_0")
trainer.patch.init_lora("lora_1")
trainer.patch.set_trainable_adapters(["lora_0"])

In [ ]:
model.model.layers[0].self_attn.o_proj.lora_A.keys()

In [ ]:
!nvidia-smi
model(**batch)
!nvidia-smi

1 LoRA
```
116: 9079MiB -> 9839MiB (760)
241: 9141MiB -> 9933MiB (792, ~0.17MiB/tok)
346: 9205MiB -> 10093MiB (888, ~0.39MiB/tok)
461: 9125MiB -> 10128MiB (1003MiB, ~0.5MiB/tok)
4601: 9139MiB -> 16176MiB (7037MiB, ~1.36 MiB/tok)
```

In [ ]:
(792-750)/241

In [ ]:
raise ValueError()

In [ ]:
10128-9125

In [ ]:
batches=[
    batch
    for _ in range(5000)
]

In [ ]:
trainer.train(batches)

In [ ]:
weights = {}
for name, module in model.named_modules():
    if isinstance(module, BitLinearWithLoRA):
        weight = module.get_bitnet_weight().detach().cpu()
        weights[f"{name}.weight"] = weight

In [ ]:
weights["model.layers.0.self_attn.o_proj.weight"].shape

In [ ]:
import gc

del model, trainer
gc.collect()
torch.cuda.empty_cache()

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map=DEVICE,
)

In [ ]:
weights["model.layers.0.self_attn.o_proj.weight"].dtype

In [ ]:
model.load_state_dict(
    dict(model.state_dict(), **weights),
    strict=False
)

In [ ]:
set(weights["model.layers.0.self_attn.o_proj.weight"].float().numpy().ravel())

In [ ]:
set( (model.state_dict()["model.layers.0.self_attn.o_proj.weight"] * 1).detach().cpu().float().numpy().ravel() )

In [ ]:
check_generation(model, tokenizer)